# Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import pandas as pd
import numpy as np
import time
import jsonlines

In [13]:
%load_ext autoreload
%autoreload 2

from seewhence import models
from seewhence import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Directories & Paths
OUTPUT_DIR = 'data'

# Load and split Sarcasm Data

In [4]:
# reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
# twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'
# train_split = 0.8
# %run ./preprocessing.ipynb

# Torch Text Dataset

In [53]:
# Fields
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
fields = {'combined': ('text', TEXT), 'label': ('label', LABEL)}

In [54]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = OUTPUT_DIR,
                                        train = 'train.csv',
                                        validation ='valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields
)

## Build Vocab

In [55]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.twitter.27B.200d", 
                 unk_init = torch.Tensor.normal_)

In [56]:
LABEL.build_vocab(train_data)

# Iterator

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# LSTM

In [105]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.6
lstm_layers=2
output_path = 'models_lstm_'


model = models.LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            num_linear=N_LAYERS,  
            dropout=DROPOUT,
            lstm_layers = lstm_layers)

In [106]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [107]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [108]:
N_EPOCHS = 8

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train.train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = train.evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = train.epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        if output_path:
            torch.save(model.state_dict(), output_path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss: 0.668 | Train Acc: 57.97%
	 Val. Loss: 0.665 |  Val. Acc: 51.40%
Epoch: 02 | Epoch Time: 0m 5s
	Train Loss: 0.616 | Train Acc: 66.43%
	 Val. Loss: 0.635 |  Val. Acc: 66.70%
Epoch: 03 | Epoch Time: 0m 5s
	Train Loss: 0.533 | Train Acc: 74.05%
	 Val. Loss: 0.629 |  Val. Acc: 66.66%
Epoch: 04 | Epoch Time: 0m 5s
	Train Loss: 0.441 | Train Acc: 80.18%
	 Val. Loss: 0.720 |  Val. Acc: 64.73%
Epoch: 05 | Epoch Time: 0m 5s
	Train Loss: 0.344 | Train Acc: 86.64%
	 Val. Loss: 0.766 |  Val. Acc: 67.43%
Epoch: 06 | Epoch Time: 0m 5s
	Train Loss: 0.259 | Train Acc: 90.66%
	 Val. Loss: 0.832 |  Val. Acc: 66.02%
Epoch: 07 | Epoch Time: 0m 5s
	Train Loss: 0.176 | Train Acc: 94.37%
	 Val. Loss: 1.062 |  Val. Acc: 65.45%
Epoch: 08 | Epoch Time: 0m 5s
	Train Loss: 0.141 | Train Acc: 95.64%
	 Val. Loss: 1.050 |  Val. Acc: 65.60%


In [103]:
model.load_state_dict(torch.load(output_path))
test_loss, test_acc = train.evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.615 | Test Acc: 65.41%
